<a href="https://colab.research.google.com/github/arikaran007/ML-Prediction/blob/main/Packet_classification_and_route_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers, Input, Model
import tensorflow as tf

In [2]:
packet_data = pd.read_csv('/content/drive/MyDrive/generated_data.csv')
packet_data.head()

,SourceIP,DestIP,SourcePort,DestPort,Protocol,Label
0,44.230.34.47,190.169.240.75,14974,13601,TCP,Web Traffic
1,11.92.116.173,7.194.76.181,54333,14741,TCP,SSH
2,54.226.136.254,189.203.211.191,58746,63176,TCP,SSH
3,125.244.197.253,80.8.223.191,31450,2070,TCP,HTTPS
4,106.215.234.247,89.145.105.105,17090,28306,UDP,HTTPS


In [3]:
route_data = pd.read_csv('/content/drive/MyDrive/route_optimization_dataset.csv')
route_data.head()

,Source,Destination,Traffic_Type,Current_Route,Link_Capacity,Latency
0,RouterB,Server2,VideoStreaming,C,571,7
1,RouterA,Switch1,VideoStreaming,C,958,4
2,Switch1,Server2,WebTraffic,B,793,2
3,RouterB,Server2,VoIP,B,521,12
4,Server1,Switch1,VideoStreaming,C,669,16


In [48]:
X_packet = packet_data.drop(['Label', 'Protocol'], axis=1)
y_packet = LabelEncoder().fit_transform(packet_data['Label'])
X_packet_train, X_packet_test, y_packet_train, y_packet_test = train_test_split(X_packet, y_packet, test_size=0.2, random_state=42)
y_packet

array([3, 2, 2, 1, 1, 1, 0, 0, 2, 3, 1, 2, 3, 2, 0, 3, 2, 0, 2, 2, 1, 2,
       3, 0, 0, 3, 3, 1, 3, 0, 0, 3, 3, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 2,
       3, 2, 3, 0, 2, 2, 1, 0, 0, 3, 1, 1, 3, 0, 2, 3, 3, 1, 0, 3, 2, 0,
       3, 0, 3, 1, 1, 1, 2, 2, 3, 2, 2, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 3,
       3, 3, 2, 3, 3, 3, 0, 2, 1, 1, 2, 1, 3, 2, 3, 0, 2, 3, 0, 1, 1, 1,
       3, 0, 0, 3, 0, 2, 1, 1, 2, 3, 1, 0, 2, 3, 3, 0, 0, 1, 3, 3, 3, 1,
       3, 3, 0, 1, 2, 1, 3, 3, 3, 1, 3, 2, 2, 2, 1, 3, 3, 3, 3, 3, 0, 3,
       0, 3, 2, 2, 0, 1, 1, 3, 3, 3, 0, 1, 2, 3, 2, 3, 1, 0, 3, 3, 3, 0,
       3, 2, 2, 1, 3, 3, 1, 0, 2, 3, 1, 3, 0, 2, 1, 0, 1, 1, 2, 0, 0, 2,
       1, 0, 3, 3, 0, 0, 3, 3, 3, 1, 2, 1, 2, 0, 0, 3, 1, 2, 3, 2, 1, 1,
       3, 3, 1, 1, 1, 2, 0, 3, 1, 2, 2, 2, 1, 3, 3, 3, 3, 1, 3, 2, 3, 1,
       0, 3, 3, 1, 3, 1, 3, 2, 3, 2, 1, 2, 2, 2, 1, 3, 1, 3, 1, 0, 1, 0,
       3, 1, 0, 2, 0, 1, 2, 1, 1, 3, 3, 1, 1, 2, 3, 3, 2, 0, 3, 2, 0, 2,
       1, 0, 0, 0, 0, 0, 1, 1, 3, 2, 2, 0, 1, 2, 3,

In [9]:
categorical_columns_packet = ['SourceIP', 'DestIP']
categorical_columns_packet

['SourceIP', 'DestIP']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

encoder_packet = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_packet_train_encoded = encoder_packet.fit_transform(X_packet_train[categorical_columns_packet])

In [7]:
X_route = route_data.drop(['Traffic_Type', 'Current_Route'], axis=1)
y_route = LabelEncoder().fit_transform(route_data['Traffic_Type'])
X_route_train, X_route_test, y_route_train, y_route_test = train_test_split(X_route, y_route, test_size=0.2, random_state=42)

In [10]:
categorical_columns_route = ['Source', 'Destination']
categorical_columns_route

['Source', 'Destination']

In [11]:
encoder_route = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_route_train_encoded = encoder_route.fit_transform(X_route_train[categorical_columns_route])
X_route_train_encoded

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [12]:
packet_input = Input(shape=(X_packet_train_encoded.shape[1],), name='input_packet')
hidden_layer1_packet = layers.Dense(64, activation='relu')(packet_input)
hidden_layer2_packet = layers.Dense(32, activation='relu')(hidden_layer1_packet)
output_layer_packet = layers.Dense(len(set(y_packet)), activation='softmax', name='output_packet')(hidden_layer2_packet)

In [15]:
packet_fnn_model = Model(inputs=packet_input, outputs=output_layer_packet)


In [16]:
route_input = Input(shape=(X_route_train_encoded.shape[1],), name='input_route')
hidden_layer1_route = layers.Dense(64, activation='relu')(route_input)
hidden_layer2_route = layers.Dense(32, activation='relu')(hidden_layer1_route)
output_layer_route = layers.Dense(len(set(y_route)), activation='softmax', name='output_route')(hidden_layer2_route)

In [17]:
route_fnn_model = Model(inputs=route_input, outputs=output_layer_route)

In [18]:
merged = layers.concatenate([packet_fnn_model.output, route_fnn_model.output])

In [20]:
joint_layer = layers.Dense(32, activation='relu')(merged)
joint_layer

<KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'dense_5')>

In [21]:
final_output = layers.Dense(len(set(y_packet)), activation='softmax', name='final_output')(joint_layer)

In [22]:
joint_model = Model(inputs=[packet_fnn_model.input, route_fnn_model.input], outputs=final_output)

In [23]:
joint_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy']
                    )

In [24]:
dataset_packet = tf.data.Dataset.from_tensor_slices((
    {"input_packet": X_packet_train_encoded.astype(float)},
    tf.constant(y_packet_train, dtype=tf.float32)
))

In [25]:
dataset_route = tf.data.Dataset.from_tensor_slices((
    {"input_route": X_route_train_encoded.astype(float)},
    tf.constant(y_route_train, dtype=tf.float32)
))

In [26]:
joint_model.fit(
    {"input_packet": X_packet_train_encoded.astype(float), "input_route": X_route_train_encoded.astype(float)},
    tf.constant(y_packet_train, dtype=tf.float32),
    epochs=100,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/100
20/20 [==============================] - 3s 32ms/step - loss: 1.4273 - accuracy: 0.2156 - val_loss: 1.3934 - val_accuracy: 0.2250
Epoch 2/100
20/20 [==============================] - 0s 12ms/step - loss: 1.3768 - accuracy: 0.3281 - val_loss: 1.3659 - val_accuracy: 0.3625
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 1.3507 - accuracy: 0.3875 - val_loss: 1.3598 - val_accuracy: 0.3625
Epoch 4/100
20/20 [==============================] - 0s 15ms/step - loss: 1.3376 - accuracy: 0.3875 - val_loss: 1.3590 - val_accuracy: 0.3625
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 1.3232 - accuracy: 0.3875 - val_loss: 1.3616 - val_accuracy: 0.3625
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 1.2882 - accuracy: 0.3875 - val_loss: 1.3660 - val_accuracy: 0.3625
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 1.2057 - accuracy: 0.3875 - val_loss: 1.3808 - val_accuracy: 0.3625
Epoch 

In [58]:
# Assuming you have new data for packet and route
# Make sure to preprocess the new data similar to your training data

# Example new data (adjust according to your actual data)
new_packet_data = pd.DataFrame({
    'SourceIP': ['125.244.197.253'],
    'DestIP': ['80.8.223.191']
})

new_route_data = pd.DataFrame({
    'Source': ['RouterA'],
    'Destination': ['Switch1']
})

# Preprocess new data
new_packet_encoded = encoder_packet.transform(new_packet_data[categorical_columns_packet])
new_route_encoded = encoder_route.transform(new_route_data[categorical_columns_route])

# Make predictions using the joint model
predictions = joint_model.predict({
    'input_packet': new_packet_encoded.astype(float),
    'input_route': new_route_encoded.astype(float)
})



1/1 [==============================] - 0s 39ms/step


In [59]:
# Mapping index to labels
label_mapping = {0: 'Video Streaming', 1: 'HTTPS', 2: 'SSH', 3: 'Web Traffic'}

# Assuming predictions is a NumPy array containing predicted probabilities
predicted_indices = np.argmax(predictions, axis=1)

# Map indices to labels
predicted_labels = [label_mapping[idx] for idx in predicted_indices]

# Print the predicted labels
print(predicted_labels)

# Mapping index to Traffic_Type labels
traffic_type_mapping = {0: 'VideoStreaming', 1: 'VOIP', 2: 'Web Traffic', 3: 'FTP'}






['HTTPS']
